<a href="https://colab.research.google.com/github/ImaginationX4/Path_to_MARL/blob/master/4_Human_level_control_through_deep_reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium[classic-control]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.6 MB/s eta 0:00:00


# Frozen-Lake

We use colab tp try our neural netwerk, for more details check the corresponding  jupyter note book.


In [ ]:
import gymnasium as gym
import numpy as np
from collections import deque
import random
import torch
from torch import nn
import torch.nn.functional as F

class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.layer_int = nn.Linear(state_size, 64)
        self.layer_out = nn.Linear(64, action_size)
    def forward(self, inputs):
        q_values = self.layer_int(inputs)
        q_values = self.layer_out(q_values)
        #action = torch.argmax(q_values).item()
        return q_values#.detach().numpy()
def state_to_dqn_input( state:int, num_states:int)->torch.Tensor:
        input_tensor = torch.zeros(num_states)
        input_tensor[state] = 1
        return input_tensor

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model_1=DQN(16,4)
# Check model device
next(model_1.parameters()).device

Using device: cuda


device(type='cpu')

In [ ]:
#parameter
EPOCH=10000
learning_rate_a = 0.001
discount_factor_g = 0.9
network_sync_rate = 10
mini_batch_size = 32
epsilon = 1
#enviorment
env= gym.make("FrozenLake-v1",is_slippery=False)
#experience buffer (s_i,a,r,s_i+1,done)
memory = deque([], maxlen=100)
terminated = False
truncated  = False
reward_count = 0
observation, info = env.reset(seed=42)
while(reward_count==0):
    action = env.action_space.sample()
    # add to memory
    new_observation, reward, terminated, truncated, info = env.step(action)
    observation_t = state_to_dqn_input(observation, 16)
    new_observation_t = state_to_dqn_input(new_observation, 16)
    memory.append((observation_t, action , reward, new_observation_t,terminated))
    observation = new_observation
    reward_count+=reward
    if terminated or truncated:
        observation, info = env.reset()
#double model,loss,optimizer
loss_fn = nn.MSELoss()
train_model = DQN(16,4).to(device)
target_model = DQN(16,4).to(device)
target_model.load_state_dict(train_model.state_dict())
optimizer  = torch.optim.Adam(train_model.parameters(), lr=learning_rate_a)
#epsilon is combined with epochs
epochs = np.linspace(0,1,EPOCH)
for i in range(EPOCH):
  observation, info = env.reset(seed=42)
  terminated = False
  truncated  = False
  while(not terminated and not truncated):
    if random.random() < epsilon:
        # select random action
        action = env.action_space.sample() # actions: 0=left,1=down,2=right,3=up
    else:
        with torch.no_grad():
            action = train_model(state_to_dqn_input(observation, 16).to(device)).argmax().item()
    new_observation, reward, terminated, truncated, info = env.step(action)
    observation_t = state_to_dqn_input(observation, 16)
    new_observation_t = state_to_dqn_input(new_observation, 16)
    observation = new_observation

    memory.append((observation_t, action , reward, new_observation_t,terminated))
  if i%1000==0:
      print('got the gift!'+f'{i} times')

  mini_barch = random.sample(memory, mini_batch_size)

  obs_a = np.asarray([t[0] for t in mini_barch])
  action_a = np.asarray([t[1] for t in mini_barch])
  reward_a = np.asarray([t[2] for t in mini_barch])
  new_obs_a = np.asarray([t[3] for t in mini_barch])
  terminated_a = np.asarray([t[4] for t in mini_barch])

  obs_t = torch.as_tensor(obs_a,dtype=torch.float32).to(device)
  action_t = torch.as_tensor(action_a,dtype=torch.int64).view(-1,1).to(device)
  reward_t = torch.as_tensor(reward_a,dtype=torch.float32).to(device).view(-1,1)
  new_obs_t = torch.as_tensor(new_obs_a,dtype=torch.float32).to(device)
  terminated_t = torch.as_tensor(terminated_a,dtype=torch.float32).to(device).view(-1,1)
  #compute the target
  max_q_action = target_model(new_obs_t).max(dim=1,keepdim=True)[0]
  target = reward_t+ (1-terminated_t)*discount_factor_g*max_q_action
  #compute the loss
  #print(action_t)
  q_values = train_model(obs_t).gather(dim=-1,index=action_t)
  loss = F.smooth_l1_loss(q_values,target)
  #optimizer
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  #update
  epsilon = max(epsilon - 1/EPOCH, 0)
  if i % network_sync_rate==0:
      target_model.load_state_dict(train_model.state_dict())
env.close()
torch.save(train_model.state_dict(), "frozen_lake_dql.pt")

In [ ]:
a = state_to_dqn_input(2, 16).to(device)
train_model(a)

# Let's try Cart Pole!

Action: left/right 0/1 <br>
Observation: Cart Position, Cart Velocity, Pole Angle, Pole Angular Velocity



In [ ]:
#netwerk
class deep_Q(nn.Module):
    def __init__(self):
        super().__init__()
        self.net= nn.Sequential(
            nn.Linear(4,64),
            nn.Tanh(),
            nn.Linear(64,2)
        )

    def forward(self,x):
        return self.net(x)



replay_buffer

In [ ]:
replay_buffer= deque([], maxlen=100)
env= gym.make('CartPole-v1')
observation, info = env.reset(seed=42)
for _ in range(100):
    #1
    action =env.action_space.sample()

    #2

    observation_next, reward, terminated, truncated, info = env.step(action)
    states =(observation,action,observation_next,reward,terminated)
    replay_buffer.append(states)
    observation = observation_next
    #print(observation_next)

    if terminated or truncated:
        observation, info = env.reset()
env.close()

Training

In [ ]:
EPOCH =10000
learning_rate_a = 0.001
discount_factor_g = 0.9
network_sync_rate = 10
mini_batch_size = 32
epsilon = 1
train_model = deep_Q().to(device)
target_model = deep_Q().to(device)
optimizer  = torch.optim.Adam(train_model.parameters(), lr=learning_rate_a)
for i in range(EPOCH):
  observation, info = env.reset(seed=42)
  terminated = False
  truncated  = False
  while(not terminated or not truncated):
    #1
    if random.random()<epsilon:
      action =env.action_space.sample()
    else:
      with torch.no_grad():
        action = train_model(torch.tensor(observation,dtype=torch.float32).to(device)).argmax().item()
    #2
    observation_next, reward, terminated, truncated, info = env.step(action)
    states =(observation,[action],observation_next,[reward],[terminated])
    replay_buffer.append(states)
    observation = observation_next

  mini_barch = random.sample(replay_buffer, mini_batch_size)
  obs_n = np.asarray([t[0] for t in mini_barch])
  action_n = np.asarray([t[1] for t in mini_barch])
  obs_next_n = np.asarray([t[1] for t in mini_barch])
  reward_n = np.asarray([t[1] for t in mini_barch])
  terminated_n = np.asarray([t[1] for t in mini_barch])

  obs_t = torch.as_tensor(obs_n,dtype=torch.float32).to(device)
  action_t = torch.as_tensor(action_n,dtype=torch.int64).to(device)
  obs_next_t = torch.as_tensor(obs_next_n,dtype=torch.float32).to(device)
  reward_t = torch.as_tensor(reward_n,dtype=torch.float32).to(device)
  terminated_t = torch.as_tensor(terminated_n,dtype=torch.float32).to(device)

  max_q_values = target_model(obs_t).max(-1,keepdim=True)[0]
  target = reward_t + (1-terminated_t)*discount_factor_g*max_q_values
  if i % 100==0:
    print(f'{i}times')

  q_values = train_model(obs_t).gather(dim=-1,index=action_t)
  loss = F.smooth_l1_loss(q_values,target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  #update
  epsilon = max(epsilon - 1/EPOCH, 0)
  if i % network_sync_rate==0:
      target_model.load_state_dict(train_model.state_dict())
env.close()
torch.save(train_model.state_dict(), "cart_Pole_dql.pt")

0times
100times
200times
300times
400times
500times
600times
700times
800times
900times
1000times
1100times
1200times
1300times
1400times
1500times
1600times
1700times
1800times
1900times
2000times
2100times
2200times
2300times
2400times
2500times
2600times
2700times
2800times
2900times
3000times
3100times
3200times
3300times
3400times
3500times
3600times
3700times
3800times
3900times
4000times
4100times
4200times
4300times
4400times
4500times
4600times
4700times
4800times
4900times
5000times
5100times
5200times
5300times
5400times
5500times
5600times
5700times
5800times
5900times
6000times
6100times
6200times
6300times
6400times
6500times
6600times
6700times
6800times
6900times
7000times
7100times
7200times
7300times
7400times
7500times
7600times
7700times
7800times
7900times
8000times
8100times
8200times
8300times
8400times
8500times
8600times
8700times
8800times
8900times
9000times
9100times
9200times
9300times
9400times
9500times
9600times
9700times
9800times
9900times
